![](https://www.saa-authors.eu/picture/739/ftw_768/saa-mtcwmza4nzq5mq.jpg)

## Plan for the day

* Mircea: Self Intro
* Research Prototype Demo: FMD
* Topic: Logging
* (Mini-)Topic: Service Level Agreements
* Assignment
* Group Presentation: ?
   

# Who Am I

Currently: Associate Professor ITU

Before: 
  - U. Groningen (NL)
  - U. Bern (CH)
  - IBM TJ Watson (US)
  - U. Lugano (CH)

Research Interests:
  - Software **Evolution**
  - Tools for Software **Maintenance**
  - Mining Software Repositories


# OPs Experience

OPing a research prototype
- for intermediate language learners
- ~100 users per week



<img src="images/zeeguu_context.jpg" alt="Drawing" style="width: 300px;"/>


- Demo: [zeeguu.org](https://zeeguu.org) 
- Invite Code: zeeguu-beta
- Open Source: https://github.com/zeeguu-ecosystem/

# Monitoring - Brief Looking Back


### Question #1 for You: Which of your endpoints is the slowest? How slow is it?

Let's see who can get this information first!


### Question #2 for You: Where is the time being spent in this endpoint?

Does your monitoring infra allow you to answer this question?

## Flask Monitoring Dashboard

Research Project 

Application Performance Monitor for Python + Flask

Goals:
* Very easy to deploy & use
* Version-control-aware

Open source, MIT at: https://github.com/flask-dashboard/

# FMD: Demo

- Demo: https://fmd-master.herokuapp.com/
- Documentation: https://github.com/flask-dashboard/Flask-MonitoringDashboard/

# Observations

* Aimed at small projects (stand-alone API / App)


* Can be easily installed because its laser-focused: Flask/Python


* Profiling introduces significant overhead
  - monitoring levels deployable with fine granularity
  - measuring the performance is tricky


# Meta-Observations

* Started as a bachelor thesis continued as master's thesis
  - Stick to your project (it takes a years till something becomes successful)
  - There are still several theses in there...


* Try it if you have a Flask API/App

## Limitations of Monitoring


* Tracks high-level metrics the system (error rate of endpoint: 2%)
* Does not explain WHY there was a problem 

For the WHY we need more detailed information: 

  - profiling (as we saw in FMD)
  - or **logging** as we will discuss today
  - or tracing, which we don't discuss today
         
         
Read More: [Why Grafana is Good at Metrics and Not Logs](https://grafana.com/blog/2016/01/05/logs-and-metrics-and-graphs-oh-my/)

# Logging


Logs are **the stream of aggregated, time-ordered events** collected from the output streams of all running processes and backing services


Purpose

* Understanding (how is your system being used?)


* Diagnosis (of an actual problem)


* Audit trails (sometimes they're legally required)
  - Anybody has experience with this?

## Logfiles 


In server-based environments they are commonly written to a file on disk (a “logfile”)

... but this is only an output format

    e.g. /var/log/auth.log
    
<img src="images/auth_log.png" alt="Drawing" style="width: 800px;"/>


A reminder of the importance of security...


## Different programs have different log formats

#### /var/log/auth.log

    Mar 16 07:15:55 zeeguu-amsterdam sshd[29424]: Invalid user vultr from 144.217.243.216 port 56450


#### /var/log/apache2/error.log

    [Wed Mar 18 20:39:02.962354 2020] [wsgi:error] [pid 18:tid 140056344164096] [remote 212.187.36.136:57046] Session is retrived from cookies
    
#### nginx

    66.249.65.62 - - [06/Nov/2014:19:12:14 +0600] "GET /?q=%E0%A6%A6%E0%A7%8B%E0%A7%9F%E0%A6%BE HTTP/1.1" 200 4356 "-" "Mozilla/5.0 (compatible; Googlebot/2.1;)"

#### /var/log/system.log (on mac)

    Mar 18 21:25:16 Harlequin logd[85]: #DECODE failed to resolve UUID: [pc:0x7fff75485ac7 ns:0x06 type:0x82 flags:0x8208 main:A52374C3-0F9D-3062-A636 pid:435]



## Syslog: What and How to Log? 

Syslog is a protocol 
* developed in 1980 
* aims at standardizing the way logs are formatted
* not only for Linux, but for any system exchanging logs.


Designed to enable the separation between:
  - sender
  - collector, and 
  - transport
    


More on syslog
- https://tools.ietf.org/html/rfc3164 (2001, cool intro, more humane)
- https://tools.ietf.org/html/rfc5424 (2009)


### Example Syslog Entry 

<img src="images/syslog_line.png" alt="Drawing" style="width: 400px;"/>

### Log Levels (Syslog)

<img src="images/syslog_levels.png" alt="Drawing" style="width: 700px;"/>





# Application Level Logging

A few principles:
- Don't worry about storage
- Log only what's necessary
- Use log levels



### A Process Should Not Worry About Storage

Don't bother to decide which logfile to write to...


If each process **writes to its unbuffered stdout stream** ...


* During development: dev looks at the terminal


* During deployment output from process is routed where needed by the OPS


* Different contexts result in different logfiles 
  - e.g. cronjob vs stand-alone



### A process should log only what is necessary

* Log files can become huge fast


* Imagine 10 servers x 1000 req / sec * 1KB per Event (you'd need quite a network bandwidth...)


Even in the context of zeeguu logfiles grew to > 2GB
- sufficient to fill the disk
- to the point of not being able to run bash commands!!!
  - Hint: ssh user@server rm ... 





### Use log levels to allow controlling the amount of output

* Allows the user of the app / library to control the amount of logging


* Better than print 
  - intention revealing
  - can be turned off gradually
  


See following example...




In [46]:
import sys
from importlib import reload 
import logging
reload(logging) #hack because of jupyter

logging.basicConfig(
        format="%(asctime)-15sZ %(levelname)s [%(module)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S.%f",
        level=logging.INFO,
        stream=sys.stdout
)

logging.debug("Got here!")
logging.info("Demo started.")
logging.warning("Ooops. Something went wrong!")
logging.error("Logging is broken in Jupyter")
logging.critical("System will shot down")


2020-03-19 00:40:56.f INFO [<ipython-input-46-86d67c93cc97>] Demo started.
2020-03-19 00:40:56.f WARNING [<ipython-input-46-86d67c93cc97>] Ooops. Something went wrong!
2020-03-19 00:40:56.f ERROR [<ipython-input-46-86d67c93cc97>] Logging is broken in Jupyter
2020-03-19 00:40:56.f CRITICAL [<ipython-input-46-86d67c93cc97>] System will shot down


# ELK: Modern, Scalable, User-Friendly Logging

One of the most popular solutions at the moment

<img src="images/elk.png" alt="Drawing" style="width: 600px;"/>


Stands for:
* ElasticSearch
* Logstash 
* Kibana


## ElasticSearch - Scalable Full Text Search

Solution based on JSON over HTTP

* Based on Apache Lucene
* Distributed & Replicated
* Almost real time full text search
* Stores logs in dedicated log indexes

Breaking News! Student project comparing MySQL with ES: 
- Did you know that MySQL has [full text search](https://dev.mysql.com/doc/refman/8.0/en/fulltext-search.html)? 
- Guess who wins?



## Logstash - Log Parser 


Converts from various log line formats to JSON

Tails log files and emits events when a new log message is added

Comes with a powerful pattern parsing plugin (Grok)

<img src="images/logstash_example_.png" alt="Drawing" style="width: 900px;"/>


Very very slow

- (Probably the reason for which ti's now possible to circumvent it via FileBeats)


## Kibana


Powerful visualization tool tailored for ElasticSearch

Available on http://localhost:5601

## Logstash Configured to Listen to Port 5000


    input {
    	tcp {
    		port => 5000
    	}
    }    

    filter {
    	grok {
    		match => { "message" => "%{TIMESTAMP_ISO8601:timestamp} %{DATA:level} %{DATA:process} %{GREEDYDATA:log}" }
    	}    

    }    

    output {
    	elasticsearch {
    		hosts => "elasticsearch:9200"
    		user => "elastic"
    		password => "changeme"
    	    index => "port_5000"
    	}
    }    






## Logstash Configured to Read Syslog from Logfile


    input {
    	file {
    		path => "/Users/mircea/local/projects/zeeguu/CodeBase/Zeeguu-Mono-Web/zeeguu_mono_web.log"
    	}
    }    

    filter {
    	grok {
    		match => { "message" => "%{TIMESTAMP_ISO8601:timestamp} %{DATA:level} %{DATA:process} %{GREEDYDATA:log}" }
    	}
        date {
            match => [ "timestamp" , "dd/MMM/yyyy:HH:mm:ss Z" ]
        }

    }    

    output {
    	elasticsearch {
    		hosts => "elasticsearch:9200"
    		user => "elastic"
    		password => "changeme"
    	    index => "zeeguu_web"
    	}
    }    







## Resources


Installing ELK with Docker: https://logz.io/blog/elk-stack-on-docker/

Good patterns for Grok: https://qbox.io/blog/logstash-grok-filter-tutorial-patterns

This one uses Filebeat: https://www.elastic.co/guide/en/logstash/current/advanced-pipeline.html

# Ethical Aspects

* Privacy and security concerns
* Be aware of who has access to the data and the data that is logged
  - If you log user details, and they ask you to "remove" them because GDPR? 




# Service Level Agreements (SLAs)

Commitment between a service provider and a client. 

Particular aspects of the service are agreed upon

* quality
* availability
* responsibilities 




# SLA Metrics

Common metrics for web apps / services

- Uptime/availability (usually percentage of all time)
- Mean response time (average time before answer)
- Mean time to recover (time to recover after outage)
- Failure frequency (number of failures/timeouts over time)






# SLA @ Google vs. Microsoft

https://cloud.google.com/translate/sla

https://azure.microsoft.com/en-us/support/legal/sla/machine-learning-service/v1_0/

What do you see? 

Which one is more fair?


-----------


# For Next Week

<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=20%/>

  1. Add logging to your system
  2. Write an SLA for your API

## 1) Add logging to your system

Deploy the ELK stack and start logging the important events in your system

0. Add logging to your codebase (change existing `print` statements)
1. Clone the https://github.com/deviantony/docker-elk.git repo
2. Change the filter correspondingly (use http://grokdebug.herokuapp.com/ if you need to try out stuff)
3. Point logstash to your logfile(s)

Make sure that you track relevant events. 
We will try to generate special events to see if you catch them.





## 2) Write an SLA 

Make sure that it's the best you can offer, but at the same time, try to aim for something that's realistic 

We will announce the ones that are the best